In [1]:
import pyparsing as pp
import re

from itertools import chain

In [2]:
number_regexp = r"[0-9]+(?:\.[0-9]+)?s?"
alpha_regexp =   r"(?!?:THUMB-)?(?:IX-|POSS-|SELF-))[A-Z0-9](?:[A-Z0-9'-]*[A-Z0-9])?(?:\.|:[0-9])?"
lookahead_regexp = r"(?:(?! [a-z])|(?= wg))"
word_all_regexp = r"(?: %s | %s )%s"% (number_regexp, alpha_regexp, lookahead_regexp)

In [3]:
handshapelist = [
  '1',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '25',
  'A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H-U',
  'I',
  'K',
  'L',
  'M',
  'N',
  'O',
  'R',
  'S',
  'T',
  'U',
  'V',
  'W',
  'X',
  'Y',
  'C-L',
  'U-L',
  'B-L',
  'P-K',
  'Q-G',
  'L-X',
  'I-L-Y',
  '5-C',
  '5-C-L',
  '5-C-tt',
  'alt-M',
  'alt-N',
  'alt-P',
  'B-xd',
  'baby-O',
  'bent-1',
  'bent-B',
  'bent-B-L',
  'bent-horns',
  'bent-L',
  'bent-M',
  'bent-N',
  'bent-U',
  'cocked-8',
  'cocked-F',
  'cocked-S',
  'crvd-5',
  'crvd-B',
  'crvd-flat-B',
  'crvd-L',
  'crvd-sprd-B',
  'crvd-U',
  'crvd-V',
  'fanned-flat-O',
  'flat-B',
  'flat-C',
  'flat-F',
  'flat-G',
  'flat-O-2',
  'flat-O',
  'full-M',
  'horns',
  'loose-E',
  'O-2-horns',
  'open-8',
  'open-F',
  'sml-C-3',
  'tight-C-2',
  'tight-C',
  'X-over-thumb'
]

In [4]:
cl_prefix = pp.one_of(["CL", "DCL", "LCL", "SCL", "BCL", "BPCL", "PCL", "ICL"])
ns_prefix = pp.Literal("ns-")
fs_prefix = pp.Literal("fs-")
lex_exceptions = pp.one_of(["part", "WHAT"])
aspect_text = pp.Literal("aspect")
index_core_ix = pp.Literal("IX-")
other_index_core = pp.one_of(["POSS-", "SELF-"])
handshape = pp.one_of(handshapelist)
person = pp.one_of(["1p", "2p", "3p"])
dash = pp.Literal("-")
arc = pp.Literal("-arc")
loc = pp.Literal("-loc")
pl = pp.Literal("-pl")
compound = pp.Literal("+")
hashtag = pp.Literal("#")
choice = pp.Literal("/")
sym = pp.Literal(">")
par1, par2 = pp.Literal("("), pp.Literal(")")
contraction = pp.Literal("^")
colon = pp.Literal(":")
omit_quote = pp.Literal("xx")
period = pp.Literal(".")
word = pp.Regex(word_all_regexp)

In [5]:
full_grammar = pp.OneOrMore(
    cl_prefix     |
    ns_prefix     |
    fs_prefix     |
    lex_exceptions|
    person |
    aspect_text   |
    index_core_ix |
    other_index_core |
    handshape |
    dash |
    arc |
    loc |
    pl |
    compound |
    hashtag |
    choice |
    sym |
    par1 |
    par2 |
    contraction |
    colon |
    omit_quote    |
    period |
    word
)

In [6]:
BATCH_SIZE = 64
EPOCHS = 30  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 40
ENG_VOCAB_SIZE = 15000
SPA_VOCAB_SIZE = 15000
num_samples = 1400

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8
data_path = "/Users/adrianajimenez/Desktop/Downloads/REUAICT/Real-Code/2025-ASL-data/data/use_data.txt"

In [7]:
input_texts = []
target_texts = []
input_tokens = set()
target_tokens = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    input_texts.append(input_text)
    target_texts.append(target_text)
    for token in input_text:
        if token not in input_tokens:
            input_tokens.add(token)
            
            
for sent in target_texts:
    token = full_grammar.parse_string(sent)
    print(token)
   # for tok in tokens:
    #    if tok not in target_tokens:
     #       target_tokens.add(tok)


input_tokens = sorted(list(input_tokens))
#target_tokens = sorted(list(target_tokens))

print("input_tokens:", input_tokens)
print("output tokens", target_texts)
#print("target_tokens:", target_tokens)
num_encoder_tokens = len(input_tokens)
#num_decoder_tokens = len(target_tokens)

ValueError: invalid pattern ("(?: [0-9]+(?:\\.[0-9]+)?s? | (?!?:THUMB-)?(?:IX-|POSS-|SELF-))[A-Z0-9](?:[A-Z0-9'-]*[A-Z0-9])?(?:\\.|:[0-9])? )(?:(?! [a-z])|(?= wg))") passed to Regex